# One Location, Multiple Temporal Scales, Multiple Operations, Mixed Integer Linear Programming Example

## [Example 2]

This is a continuation of 'One Location, One Temporal Scale, One Operation, Linear Programming Example' [Example 1]. Refer Example 1 to learn the basics on how Energia models processes. 

In this example, we add another Process [Solar PV] and Storage [Li-ion Battery]. Technology choice is modeled using binaries. Moreover, the model is multiscale as the operational capacities are decision variables. 

## From Example 1

In [1]:
from energia import *

m = Model('example2')
m.q = Period()
m.y = 4 * m.q
m.usd = Currency()

## Resources

### Conveniently declaring components

Use m.declare(\<Object Type\>, \<list of names\>) to declare a large number of objects in one step.

In [2]:
m.declare(Resource, ['power', 'wind', 'solar'])

### Set bounds on Resource flows

Unlike wind which has bound on the total consumption, we set a daily limit on solar energy. The same bound is repeated in each quarter. The following constraints are written.

$\mathbf{cons}_{solar, network, quarter_0} \leq 100$

$\mathbf{cons}_{solar, network, quarter_1} \leq 100$

$\mathbf{cons}_{solar, network, quarter_2} \leq 100$

$\mathbf{cons}_{solar, network, quarter_3} \leq 100$

In [3]:
m.solar.consume(m.q) <= 100
m.wind.consume <= 400
m.power.release.prep(180) >= [0.6, 0.7, 0.8, 0.3]

--- General Resource Balance for solar in (l, q): initializing constraint, adding consume
    Completed in 0.00015974044799804688 seconds
--- General Resource Balance for wind in (l, y): initializing constraint, adding consume
    Completed in 6.29425048828125e-05 seconds
--- General Resource Balance for power in (l, q): initializing constraint, adding release
    Completed in 0.00024127960205078125 seconds


## Operations 

### Capacity as a variable 

Here we want the optimization problem to determine the optimal capacity. Moreover, we set binaries to avoid the lower bound being adhered to if the process is not set up. 

If the bounds are meant to be compulsory limits, skip the .x 

In [4]:
m.wf = Process()
m.wf(m.power) == -1 * m.wind
m.wf.capacity.x <= 100
m.wf.capacity.x >= 10
m.capacity.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Unlike in Example 1, where the capacity was know, capacity is a variable here. 

Moreover, the expenditure associated with operating and capacitating are different

In [5]:
m.wf.operate.prep(norm=True) <= [0.9, 0.8, 0.5, 0.7]
m.wf.capacity[m.usd.spend] == 990637 + 3354
m.wf.operate[m.usd.spend] == 49
m.operate.show(True)

--- Creating map to (wf, l, y). Mapping operate: from (wf, l, q) to (wf, l, y)
    Completed in 0.00015926361083984375 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [6]:
m.pv = Process()
m.pv(m.power) == -1 * m.solar
m.pv.capacity.x <= 100
m.pv.capacity.x >= 10

m.pv.operate.prep(norm=True) <= [0.6, 0.8, 0.9, 0.7]
m.pv.capacity[m.usd.spend] == 567000 + 872046
m.pv.operate[m.usd.spend] == 90000

--- Creating map to (pv, l, y). Mapping operate: from (pv, l, q) to (pv, l, y)
    Completed in 0.0001678466796875 seconds


### Storage Operation

energia now allows storing to require the use of other resources, example power for hydrogen cryogenic storage. 

Provide an equation similar to Process, in this case the basis is the stored resource 
If no other resource is provided, it is assumed to be the charging/discharging efficiency

Note that the following are created internally: 
1. auxilary resource  with name resource.stored 
2. charging and discharging processes as storage.charge and storage.discharge 

The parameters for each of these can be set individually, thus allowing for a wide range of modeling approaches 

In [7]:
m.lii = Storage()

m.lii(m.power) == 0.9
m.lii.capacity.x <= 100
m.lii.capacity.x >= 10

# m.lii.capacity >= 10
m.lii.capacity[m.usd.spend] == 1302182 + 41432

m.lii.inventory[m.usd.spend] == 2000
m.lii.charge.capacity <= 100
m.lii.discharge.capacity <= 100

--- General Resource Balance for power.lii in (l, y): initializing constraint, adding inventory


## Locating Operations

Operations can be located as 

operation.locate(\<list of locations\>)

or 

m.location.operations(\<list of operations\>)

They both do the same thing 

In [8]:
m.pv.locate(m.network)
m.network.locate(m.wf, m.lii)

--- General Resource Balance for power in (l, q): adding produce
    Completed in 0.000156402587890625 seconds
--- General Resource Balance for solar in (l, q): adding expend
    Completed in 8.130073547363281e-05 seconds
--- General Resource Balance for power in (l, q): adding produce
    Completed in 0.0001227855682373047 seconds
--- General Resource Balance for wind in (l, y): adding expend
    Completed in 5.745887756347656e-05 seconds
--- Assuming  power.lii inventory capacity is unbounded in (l, y)
--- Assuming inventory of power.lii is bound by inventory capacity in (l, y)
--- Assuming inventory of power.lii is bound by inventory capacity in (l, q)
--- Creating map to (power.lii, l, y). Mapping inventory: from (power.lii, l, q) to (power.lii, l, y)
    Completed in 9.655952453613281e-05 seconds
--- General Resource Balance for power.lii in (l, q): initializing constraint, adding inventory
    Completed in 0.00016546249389648438 seconds
--- Assuming operation of lii.charge is bou

## Inventory Balance

Inventory is passed on from one time period (t - 1) to the next (t) and hence features in the general resource balance for resource.stored 

In [9]:
m.show(True)

# Mathematical Program for example2

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [10]:
m.inventory.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Optimize!

In [11]:
m.show()

# Mathematical Program for example2

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [12]:
m.usd.spend.opt()

--- Creating map to (usd, l, y). Mapping spend: from (usd, l, y, capacity, wf) to (usd, l, y)
    Completed in 0.0001747608184814453 seconds
--- Mapping spend: from (usd, l, y, operate, wf) to (usd, l, y)
    Completed in 6.318092346191406e-05 seconds
--- Mapping spend: from (usd, l, y, capacity, pv) to (usd, l, y)
    Completed in 5.698204040527344e-05 seconds
--- Mapping spend: from (usd, l, y, operate, pv) to (usd, l, y)
    Completed in 5.364418029785156e-05 seconds
--- Mapping spend: from (usd, l, y, invcapacity, power.lii) to (usd, l, y)
    Completed in 5.53131103515625e-05 seconds
--- Mapping spend: from (usd, l, y, inventory, power.lii) to (usd, l, y)
    Completed in 5.269050598144531e-05 seconds
--- Generating example2.mps
--- Creating gurobi model for example2
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-01
Read MPS format model from file example2.mps
Reading time = 0.00 seconds
EXAMPLE2: 85 rows, 78 columns, 198 nonzeros
--- Optim

## Solution

### Inventory Profiles

The inventory maintained in each time period is:

In [13]:
m.inventory.sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount charged into inventory is:

In [14]:
m.show()

# Mathematical Program for example2

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

###  Function Sets

<IPython.core.display.Math object>

In [15]:
m.produce(m.power.lii, m.lii.charge.operate, m.q).sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount discharged from inventory is:

In [16]:
m.produce(m.power, m.lii.discharge.operate, m.q).sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Integer Decisions 

All the operations are setup in this case

In [17]:
m.capacity.reporting.sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [18]:
m.capacity.sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>